## Multi Label(Head, Branch) Classifier
- 하나의 Convolution 모델에서 3개의 FC Layer 브랜치를 만들어보자

In [31]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


- 기존 train data 1가지
- mask, gender, age 각각 labeling을 한 데이터프레임 3가지
- 총 4가지의 데이터프레임을 Dataset모듈에 넣을 것임

In [2]:
# # 기본 train 데이터 셋
# df = pd.read_csv('train_label.csv')
# df.head(5)

,Unnamed: 0,path,label
0,0,../input/data/train/images/000001_female_Asian...,4
1,1,../input/data/train/images/000001_female_Asian...,4
2,2,../input/data/train/images/000001_female_Asian...,4
3,3,../input/data/train/images/000001_female_Asian...,10
4,4,../input/data/train/images/000001_female_Asian...,4


In [19]:
# # wear, incorrect, normal 3가지 클래스로 변경
# # 0~5, 6~11, 12~17끼리 묶는다
# df_mask = df.copy()

# def mask_label(x):
#     if x in [0,1,2,3,4,5]:
#         return 0    # wear
#     elif x in [6,7,8,9,10,11]:
#         return 1    # incorrect
#     else:
#         return 2    # not wear

# df_mask['label'] = df_mask['label'].apply(mask_label)

# # 남성, 여성 2가지 클래스로 변경
# # [0,1,2,6,7,8,12,13,14], [3,4,5,9,10,11,15,16,17]
# df_gender = df.copy()

# def gender_label(x):
#     if x in [0,1,2,6,7,8,12,13,14]:
#         return 0    # male
#     else:
#         return 1    # female

# df_gender['label'] = df_gender['label'].apply(gender_label)

# # 청년, 중년, 장년 3가지 클래스로 변경
# # [0,3,6,9,12,15], [1,4,7,10,13,16], [2,5,8,11,14,17]
# df_age = df.copy()

# def age_label(x):
#     if x in [0,3,6,9,12,15]:
#         return 0    # young
#     elif x in [1,4,7,10,13,16]:
#         return 1    # middle
#     else:
#         return 2    # old

# df_age['label'] = df_age['label'].apply(age_label)

# # 청년, 중년, 장년 3가지 클래스로 변경
# # [0,3,6,9,12,15], [1,4,7,10,13,16], [2,5,8,11,14,17]
# df_age = df.copy()

### Dataset  정의
- mask, gender, age 별로 label 나눔

In [32]:
class MultiBranchDataset(Dataset):
    def __init__(self, df, df_mask, df_gender, df_age, transforms):
        self.df = df
        self.image_data = self.df['path']   # x data, 이미지
        self.image_label = self.df['label'] # y data, 레이블

        self.mask_label = df_mask['label']
        self.gender_label = df_gender['label']
        self.age_label = df_age['label']

        self.transform = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        img_path = self.df['path'].iloc[idx]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)
        
        dict_label = {
            'mask' : self.mask_label[idx],
            'gender' : self.gender_label[idx],
            'age' : self.age_label[idx]
        }

        return img, dict_label

#### Transform & Dataset

In [33]:
# Transform Compose
data_transform = torchvision.transforms.Compose([
    # transforms.CenterCrop(320),
    torchvision.transforms.Resize((350,350),Image.BILINEAR),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.2,0.2,0.2)),
])

# 데이터셋 준비
df = pd.read_csv('train_label.csv')
df_mask = pd.read_csv('df_mask.csv')
df_gender = pd.read_csv('df_gender.csv')
df_age = pd.read_csv('df_age.csv')


train_dataset = MultiBranchDataset(df, df_mask, df_gender, df_age, data_transform)

### DataLoader
- 여기서 3개로 나눌 필요가 없음
- 모델 train에서 loss를 따로 나눌 것임

In [34]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)


데이터로더의 label 부분을 보면 mask, gender, age별로 배치사이즈에 맞게 레이블 정보가 담아짐

In [38]:
next(iter(train_dataloader))[1]

{'mask': tensor([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 1, 2, 1, 0, 0,
         0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 2, 2,
         0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0]),
 'gender': tensor([0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
         0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
         1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1]),
 'age': tensor([1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
         2, 0, 0, 1, 2, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 2, 0, 1, 1, 0, 0,
         1, 0, 1, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1])}

#### Conv layer 빠져나오고 브랜치 만들기

In [13]:
class MultiBranchModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.base_model = torchvision.models.resnet18(pretrained=True)

        base_model = nn.Sequential(*list(self.base_model.children())[:-1])

        self.mask = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 3, bias=True)
        )
        self.gender = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 2, bias=True)
        )
        self.age = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 3, bias=True)
        )
        

    def forward(self, x):
        x = self.base_model(x)
        
        return {
            'mask': self.mask(x),
            'gender': self.gender(x),
            'age': self.age(x)
        }


In [ ]:
model = MultiBranchModel().to(device)

optimizier = torch.optim.Adam(model.parameters())

### Loss 함수 정의
- 3가지 task 각각 loss 구하고 합침

In [ ]:
def get_loss(net_output, ground_truth):
    mask_loss = F.cross_entropy(net_output['mask'], ground_truth['mask'])
    gender_loss = F.cross_entropy(net_output['gender'], ground_truth['gender'])
    age_loss = F.cross_entropy(net_output['age'], ground_truth['age'])

    loss = mask_loss + gender_loss + age_loss

    return loss, {'mask' : mask_loss, 'gender' : gender_loss, 'age' : age_loss}

#### 학습 중에 Loss 계산
- output 결과 : mask, gender, age별 예측 레이블
- target_labels : mask, gender, age별 실제 레이블

In [ ]:
output = model(image)

loss_train, _ = get_loss(output, target_labels)
total_loss += loss_train.item()


### 테스트(Evaluation) 중, mask, gender, age별 output 뽑아내기
- 최종 클래스 예측(18개)으로 변환
- mask : 0, 1, 2 (wear, incorrect, not wear)
- gender : 0, 1  (male, female)
- age : 0, 1, 2  (young, middle, old)
- class : (mask * 6) + (gender * 3) + (age)

In [ ]:
output = model(image)
pred_mask = torch.argmax(output['mask'], dim=-1)
pred_gender = torch.argmax(output['gender'], dim=-1)
pred_age = torch.argmax(output['age'], dim=-1)

pred_class = (pred_mask * 6) + (pred_gender * 3) + (pred_age)